In [1]:
import pandas as pd 
from glob import glob #檔名模式匹配(filename pattern matching)模組(Module)
import numpy as np 
import nltk #英語NLP套件
import re
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *

### 載入爬蟲文本資料 ->髒亂 還有 缺失。

In [2]:
#獲取所有EUR的新聞路徑
files = glob('./Newsdata/CurrencyNews_1*/EUR*.csv')
print(files)

['./Newsdata\\CurrencyNews_105\\EUR10501.csv', './Newsdata\\CurrencyNews_105\\EUR10502.csv', './Newsdata\\CurrencyNews_105\\EUR10503.csv', './Newsdata\\CurrencyNews_105\\EUR10504.csv', './Newsdata\\CurrencyNews_105\\EUR10505.csv', './Newsdata\\CurrencyNews_105\\EUR10506.csv', './Newsdata\\CurrencyNews_105\\EUR10507.csv', './Newsdata\\CurrencyNews_105\\EUR10508.csv', './Newsdata\\CurrencyNews_105\\EUR10509.csv', './Newsdata\\CurrencyNews_105\\EUR10510.csv', './Newsdata\\CurrencyNews_105\\EUR10511.csv', './Newsdata\\CurrencyNews_105\\EUR10512.csv', './Newsdata\\CurrencyNews_106\\EUR10601.csv', './Newsdata\\CurrencyNews_106\\EUR10602.csv', './Newsdata\\CurrencyNews_106\\EUR10603.csv', './Newsdata\\CurrencyNews_106\\EUR10604.csv', './Newsdata\\CurrencyNews_106\\EUR10605.csv', './Newsdata\\CurrencyNews_106\\EUR10606.csv', './Newsdata\\CurrencyNews_106\\EUR10607.csv', './Newsdata\\CurrencyNews_106\\EUR10608.csv', './Newsdata\\CurrencyNews_106\\EUR10609.csv', './Newsdata\\CurrencyNews_106\\EU

In [3]:
##載入資料(並選擇新聞本文欄位)

data = pd.DataFrame()

for file in files:
    #只要'date','body','source_title'這幾個欄位，編碼為ANSI
    df = pd.read_csv(file, usecols=['date','body','source_title'],
                    dtype={ 'date': str, 'body':str,'source_title':str},
                    encoding = "ANSI")
    #反轉index順序以符合時間
    df = df.reindex(index=df.index[::-1])
    #把所有df合併
    data = pd.concat([data,df],ignore_index=True)
#重設index
df_news = data.reset_index(drop=True)
df_news

,date,body,source_title
0,2016-01-01,"Unemployment continues to fall, inflation is i...",Forbes
1,2016-01-01,2015 was a year of tremendous political and ec...,Forbes
2,2016-01-01,Riverside Co. is ringing in the new year with ...,The Wall Street Journal
3,2016-01-01,Distrust in banking system hinders capital con...,Bloomberg Business
4,2016-01-01,In the 16 years since a peace treaty was signe...,International Business Times
...,...,...,...
101573,2020-12-31,"I write about energy and commodities, from ren...",Forbes
101574,2020-12-31,Philanthropic endeavors are nothing new to the...,MarketWatch
101575,2020-12-31,Wall Street indices finished 2020 at all-time ...,International Business Times
101576,2020-12-31,The U.S. dollar went from life preserver to ca...,MarketWatch


In [4]:
# 刪除遺漏值
df_news = df_news.dropna()
df_news = df_news.reset_index(drop = True)

In [5]:
df_news.shape

(101558, 3)

### 處理時間

In [6]:
#確認時間是否正確 ->2021/5 的日期格式不對。
for i in df_news["date"]:
    print(i)

2016-01-01
2016-01-01
2016-01-01
2016-01-01
2016-01-01
2016-01-01
2016-01-01
2016-01-02
2016-01-02
2016-01-02
2016-01-02
2016-01-02
2016-01-02
2016-01-02
2016-01-02
2016-01-03
2016-01-03
2016-01-03
2016-01-03
2016-01-03
2016-01-03
2016-01-03
2016-01-03
2016-01-03
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-04
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-05
2016-01-06
2016-01-06
2016-01-06
2016-01-06
2016-01-06

2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-16
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17
2016-02-17

2016-03-18
2016-03-18
2016-03-18
2016-03-18
2016-03-18
2016-03-18
2016-03-18
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-19
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-03-20
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21
2016-03-21

2016-05-03
2016-05-03
2016-05-03
2016-05-03
2016-05-03
2016-05-03
2016-05-03
2016-05-03
2016-05-03
2016-05-03
2016-05-03
2016-05-03
2016-05-03
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04
2016-05-04

2016-06-05
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-06
2016-06-07
2016-06-07
2016-06-07

...

2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-27
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28
2020-07-28

2020-08-30
2020-08-30
2020-08-30
2020-08-30
2020-08-30
2020-08-30
2020-08-30
2020-08-30
2020-08-30
2020-08-30
2020-08-30
2020-08-30
2020-08-30
2020-08-30
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-08-31
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01
2020-09-01

2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-05
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06
2020-10-06

2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-17
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18
2020-11-18

2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-24
2020-12-25
2020-12-25
2020-12-25
2020-12-25
2020-12-25
2020-12-25
2020-12-25
2020-12-25
2020-12-25
2020-12-25
2020-12-25
2020-12-25
2020-12-25
2020-12-25
2020-12-25
2020-12-26
2020-12-26
2020-12-26
2020-12-26
2020-12-26
2020-12-26
2020-12-26
2020-12-26
2020-12-26
2020-12-26
2020-12-26
2020-12-26
2020-12-26
2020-12-26
2020-12-26
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-27
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28
2020-12-28

### 把奇怪的日期處理掉

In [7]:
df_news = df_news[df_news["date"].str.contains("[0-9]*-[0-9]*-[09]*")]
df_news = df_news.reset_index(drop = True)

In [8]:
df_news

,date,body,source_title
0,2016-01-01,"Unemployment continues to fall, inflation is i...",Forbes
1,2016-01-01,2015 was a year of tremendous political and ec...,Forbes
2,2016-01-01,Riverside Co. is ringing in the new year with ...,The Wall Street Journal
3,2016-01-01,Distrust in banking system hinders capital con...,Bloomberg Business
4,2016-01-01,In the 16 years since a peace treaty was signe...,International Business Times
...,...,...,...
101553,2020-12-31,"I write about energy and commodities, from ren...",Forbes
101554,2020-12-31,Philanthropic endeavors are nothing new to the...,MarketWatch
101555,2020-12-31,Wall Street indices finished 2020 at all-time ...,International Business Times
101556,2020-12-31,The U.S. dollar went from life preserver to ca...,MarketWatch


### 處理新聞title

In [9]:
print(df_news.shape[0])
#瞭解個來源新聞分布數量
print(df_news.drop_duplicates(subset = "source_title"))

101558
             date                                               body  \
0      2016-01-01  Unemployment continues to fall, inflation is i...   
2      2016-01-01  Riverside Co. is ringing in the new year with ...   
3      2016-01-01  Distrust in banking system hinders capital con...   
4      2016-01-01  In the 16 years since a peace treaty was signe...   
6      2016-01-01  It's 2016! 4 trades for the New Year\r\n\r\nTh...   
10     2016-01-02  TAIPEI, Taiwan, Dec. 30, 2015 /PRNewswire/ -- ...   
11     2016-01-02  Summary\r\n\r\nPoor inflation, shaky business ...   
17     2016-01-03  c AFP/File Jacques DemarthonFranco-American Gu...   
96     2016-01-06  Just before Christmas, some at Volkswagen dare...   
146    2016-01-07  LONDON (Reuters) - Global shares tumbled for a...   
294    2016-01-11  What: Dominoes Pizza (NYSE:DPZ) shareholders h...   
32864  2017-10-24  台灣健保雖然目前面來醫療資源浪費、分級制度不夠完善等問題，但在全世界就醫制度來說還是相當惠民...   
79110  2020-01-01  You could call it west Africa's Brexit

In [10]:
dic = {} #資料有的tilte都加進去
news_resource = {} #只放有大於五篇以上的title
for i in df_news["source_title"]:
    if i not in dic:
        dic[i] = 1
    elif i in dic:
        dic[i] = dic[i]+1
        
for key, value in dic.items():
    if dic[key] > 5:
        news_resource[key] = value

In [11]:
#整理重複來源
news_resource

{'Forbes': 4771,
 'The Wall Street Journal': 7518,
 'Bloomberg Business': 16174,
 'International Business Times': 5636,
 'CNBC': 10369,
 'MarketWatch': 7456,
 'Seeking Alpha': 13657,
 'Business Insider': 7090,
 'Financial Times': 14070,
 'Yahoo Finance': 12612,
 'The Motley Fool': 1667,
 'Financial Times News': 537}

In [12]:
#這邊要人工判斷
news_resource['Yahoo Finance'] = news_resource['Yahoo Finance']
news_resource['Financial Times'] = news_resource['Financial Times']+news_resource['Financial Times News']
del news_resource['Financial Times News']

In [13]:
news_resource

{'Forbes': 4771,
 'The Wall Street Journal': 7518,
 'Bloomberg Business': 16174,
 'International Business Times': 5636,
 'CNBC': 10369,
 'MarketWatch': 7456,
 'Seeking Alpha': 13657,
 'Business Insider': 7090,
 'Financial Times': 14607,
 'Yahoo Finance': 12612,
 'The Motley Fool': 1667}

In [14]:
# 建立新聞來源list
new_publishment = []
for key,value in news_resource.items():
    new_publishment.append(key)
print(new_publishment)

['Forbes', 'The Wall Street Journal', 'Bloomberg Business', 'International Business Times', 'CNBC', 'MarketWatch', 'Seeking Alpha', 'Business Insider', 'Financial Times', 'Yahoo Finance', 'The Motley Fool']


In [15]:
np.save('./result/News_resource/EUR_new_resource.npy', news_resource)

#load method
# import numpy as np
# new_dict = np.load('./result/News_resource/EUR_new_resource.npy', allow_pickle='TRUE')
# print(new_dict)

In [16]:
#把df中的重複title替換掉，其餘的若不再新聞來源list則刪除
for i in range(df_news.shape[0]):
    if df_news["source_title"][i] == 'finance.yahoo.com':
        df_news["source_title"][i] = 'Yahoo Finance'
    elif df_news["source_title"][i] == 'Yahoo! Finance':
        df_news["source_title"][i] = 'Yahoo Finance'
    elif df_news["source_title"][i] == 'Financial Times News':
        df_news["source_title"][i] = 'Financial Times'
    elif df_news["source_title"][i] == 'Financial Times (FT)':
        df_news["source_title"][i] = 'Financial Times'
    elif df_news['source_title'][i] not in new_publishment:
        df_news.drop(i,inplace=True)



In [17]:
df_news = df_news.reset_index(drop =True)

In [18]:
df_news.shape

(101557, 3)

### 添加TF-IDF欄位

In [19]:
df_news.insert(2, "tfidftext", "") 
df_news.insert(2, "word_segmentation", "") 
df_news = df_news.reset_index(drop = True)
df_news

,date,body,word_segmentation,tfidftext,source_title
0,2016-01-01,"Unemployment continues to fall, inflation is i...",,,Forbes
1,2016-01-01,2015 was a year of tremendous political and ec...,,,Forbes
2,2016-01-01,Riverside Co. is ringing in the new year with ...,,,The Wall Street Journal
3,2016-01-01,Distrust in banking system hinders capital con...,,,Bloomberg Business
4,2016-01-01,In the 16 years since a peace treaty was signe...,,,International Business Times
...,...,...,...,...,...
101552,2020-12-31,"I write about energy and commodities, from ren...",,,Forbes
101553,2020-12-31,Philanthropic endeavors are nothing new to the...,,,MarketWatch
101554,2020-12-31,Wall Street indices finished 2020 at all-time ...,,,International Business Times
101555,2020-12-31,The U.S. dollar went from life preserver to ca...,,,MarketWatch


In [20]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101557 entries, 0 to 101556
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   date               101557 non-null  object
 1   body               101557 non-null  object
 2   word_segmentation  101557 non-null  object
 3   tfidftext          101557 non-null  object
 4   source_title       101557 non-null  object
dtypes: object(5)
memory usage: 3.9+ MB


In [21]:
#移除html標籤
def movehtml(text):
    dr = re.compile(r'<[^>]+>',re.S)
    return dr.sub('',text)

In [22]:
#移除文本標點以及數字
def removepunctuation(text):
    remove_chars = '[0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
    return re.sub(remove_chars, '', text)

In [23]:
#去除\n\r\t
def movebackslash(text):
    text = text.replace("\r", " ")
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    text = text.replace("  ", " ")
    text = text.replace("   ", " ")
    return text

In [24]:
#還原縮寫字
replacement_patterns = [
(r'won\'t', 'will not'),
(r'can\'t', 'cannot'),
(r'i\'m', 'i am'),
(r'ain\'t', 'is not'),
(r'(\w+)\'ll', '\g<1> will'),
(r'(\w+)n\'t', '\g<1> not'),
(r'(\w+)\'ve', '\g<1> have'),
(r'(\w+)\'s', '\g<1> is'),
(r'(\w+)\'re', '\g<1> are'),
(r'(\w+)\'d', '\g<1> would')]

class RegexpReplacer(object):
    def __init__(self, patterns=replacement_patterns):
        self.patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            (s, count) = re.subn(pattern, repl, s)
        return s
        



In [25]:
#正規化處理
replacer = RegexpReplacer()
for news_index in range(df_news.shape[0]):
    a = df_news["body"][news_index]
    b = ''.join(a) #確保a為str，而非list
    c = b.lower() #正規化，轉小寫
    result = replacer.replace(c) # 還原縮寫
    
    #斷句 ->bert用
    sentences = nltk.sent_tokenize(result)
    for i in range(len(sentences)):
        sentences[i] = removepunctuation(sentences[i]) #移除文本標點以及數字
        sentences[i] = movebackslash(sentences[i]) #去除\n\r\t
    df_news["word_segmentation"][news_index] = sentences
    
    #斷詞 -> word2vec
    result = removepunctuation(result) #移除文本標點以及數字
    result = movehtml(result) #移除html標籤
    result = movebackslash(result) #去除\n\r\t
    df_news["body"][news_index] = result
    

In [26]:
df_news['word_segmentation'][1]

[' was a year of tremendous political and economic volatility including continued stale mate of the greek debt crisis reforms the greek impact on stock markets the chinese stock market summer crash plummeting oil prices do oil prices foreshadow a slump in us stock markets',
 'and repeated terrorists attacks from paris to nigeria and lebanon',
 'yet us stock markets largely weathered the volatility the sp ended the year roughly flat within ',
 'this event has happened on only occasions since the nasdaq yet again outperformed the sp which has been the case every year since the financial crisis and closed the year at ',
 'on the economic policy front some dramatic decisions signaled change in direction and sentiment a the us federal reserve raised interest rate for the first time in years in the historic december meeting and b the agreed to add the chinese currency the renminbi to its basket of special drawing rights sdr currencies',
 'what should investors expect in ',
 'the coming year 

In [27]:
#Word Segmentation (斷詞)
for newsbody_index in range(df_news.shape[0]):
    a = df_news['body'][newsbody_index]
    newstokens = nltk.tokenize.word_tokenize(a)#將單篇文章的各句子斷詞
    df_news['tfidftext'][newsbody_index] = newstokens #將單篇文章的各句斷詞彙總到新串列，for loop 將該月所有文章集成到新串列
    

In [28]:
df_news

,date,body,word_segmentation,tfidftext,source_title
0,2016-01-01,unemployment continues to fall inflation is in...,[unemployment continues to fall inflation is i...,"[unemployment, continues, to, fall, inflation,...",Forbes
1,2016-01-01,was a year of tremendous political and econom...,[ was a year of tremendous political and econo...,"[was, a, year, of, tremendous, political, and,...",Forbes
2,2016-01-01,riverside co is ringing in the new year with i...,[riverside co is ringing in the new year with ...,"[riverside, co, is, ringing, in, the, new, yea...",The Wall Street Journal
3,2016-01-01,distrust in banking system hinders capital con...,[distrust in banking system hinders capital co...,"[distrust, in, banking, system, hinders, capit...",Bloomberg Business
4,2016-01-01,in the years since a peace treaty was signed i...,[in the years since a peace treaty was signed ...,"[in, the, years, since, a, peace, treaty, was,...",International Business Times
...,...,...,...,...,...
101552,2020-12-31,i write about energy and commodities from rene...,[i write about energy and commodities from ren...,"[i, write, about, energy, and, commodities, fr...",Forbes
101553,2020-12-31,philanthropic endeavors are nothing new to the...,[philanthropic endeavors are nothing new to th...,"[philanthropic, endeavors, are, nothing, new, ...",MarketWatch
101554,2020-12-31,wall street indices finished at alltime highs ...,[wall street indices finished at alltime highs...,"[wall, street, indices, finished, at, alltime,...",International Business Times
101555,2020-12-31,the us dollar went from life preserver to cast...,[the us dollar went from life preserver to cas...,"[the, us, dollar, went, from, life, preserver,...",MarketWatch


In [29]:
#詞性還原
for newsbody_index in range(df_news.shape[0]):
    pos = nltk.pos_tag(df_news['tfidftext'][newsbody_index])
    df_news['tfidftext'][newsbody_index] = pos

In [30]:
df_news['tfidftext'][0]

[('unemployment', 'NN'),
 ('continues', 'VBZ'),
 ('to', 'TO'),
 ('fall', 'VB'),
 ('inflation', 'NN'),
 ('is', 'VBZ'),
 ('in', 'IN'),
 ('check', 'NN'),
 ('interest', 'NN'),
 ('rates', 'NNS'),
 ('remain', 'VBP'),
 ('near', 'JJ'),
 ('historical', 'JJ'),
 ('lows', 'VBZ'),
 ('the', 'DT'),
 ('governments', 'NNS'),
 ('of', 'IN'),
 ('major', 'JJ'),
 ('us', 'PRP'),
 ('partners', 'NNS'),
 ('have', 'VBP'),
 ('no', 'DT'),
 ('intention', 'NN'),
 ('to', 'TO'),
 ('engage', 'VB'),
 ('in', 'IN'),
 ('trade', 'NN'),
 ('wars', 'NNS'),
 ('and', 'CC'),
 ('the', 'DT'),
 ('price', 'NN'),
 ('of', 'IN'),
 ('oil', 'NN'),
 ('and', 'CC'),
 ('energy', 'NN'),
 ('continues', 'VBZ'),
 ('to', 'TO'),
 ('fall', 'VB'),
 ('there', 'EX'),
 ('are', 'VBP'),
 ('no', 'DT'),
 ('major', 'JJ'),
 ('storms', 'NNS'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('forecast', 'NN'),
 ('for', 'IN'),
 ('the', 'DT'),
 ('us', 'PRP'),
 ('economy', 'NN'),
 ('nevertheless', 'RB'),
 ('very', 'RB'),
 ('few', 'JJ'),
 ('economists', 'NNS'),
 ('if', 'IN'),
 ('

In [31]:
#Lemmatization (字型還原)

for newsbody_index in range(df_news.shape[0]):
    wordnet_pos = []
    p = df_news['tfidftext'][newsbody_index]
    for word, tag in p:
        if tag.startswith('J'):
            wordnet_pos.append(nltk.corpus.wordnet.ADJ)
        elif tag.startswith('V'):
            wordnet_pos.append(nltk.corpus.wordnet.VERB)
        elif tag.startswith('N'):
            wordnet_pos.append(nltk.corpus.wordnet.NOUN)
        elif tag.startswith('R'):
            wordnet_pos.append(nltk.corpus.wordnet.ADV)
        else:
            wordnet_pos.append(nltk.corpus.wordnet.NOUN)
            
    # Lemmatizer
    lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    df_news['tfidftext'][newsbody_index] = [lemmatizer.lemmatize(p[n][0], pos=wordnet_pos[n]) for n in range(len(p))]
    

In [32]:
df_news['tfidftext'][0]

['unemployment',
 'continue',
 'to',
 'fall',
 'inflation',
 'be',
 'in',
 'check',
 'interest',
 'rate',
 'remain',
 'near',
 'historical',
 'low',
 'the',
 'government',
 'of',
 'major',
 'u',
 'partner',
 'have',
 'no',
 'intention',
 'to',
 'engage',
 'in',
 'trade',
 'war',
 'and',
 'the',
 'price',
 'of',
 'oil',
 'and',
 'energy',
 'continue',
 'to',
 'fall',
 'there',
 'be',
 'no',
 'major',
 'storm',
 'in',
 'the',
 'forecast',
 'for',
 'the',
 'u',
 'economy',
 'nevertheless',
 'very',
 'few',
 'economist',
 'if',
 'any',
 'be',
 'forecast',
 'a',
 'growth',
 'rate',
 'of',
 'more',
 'than',
 'for',
 'next',
 'year',
 'what',
 'be',
 'hold',
 'back',
 'the',
 'u',
 'economy',
 'three',
 'factor',
 'conspire',
 'against',
 'growth',
 'the',
 'uncertainty',
 'about',
 'which',
 'road',
 'will',
 'the',
 'u',
 'economy',
 'will',
 'take',
 'after',
 'the',
 'next',
 'presidential',
 'election',
 'the',
 'continue',
 'high',
 'cost',
 'of',
 'the',
 'regulatory',
 'state',
 'and'

In [33]:
#remove Stopword (停用詞) 
nltk_stopwords = nltk.corpus.stopwords.words("english")

for newsbody_index in range(df_news.shape[0]):
    p = df_news['tfidftext'][newsbody_index]
    tokens = [token for token in p if token not in nltk_stopwords]
    df_news['tfidftext'][newsbody_index] = tokens

#現在即可得到所有詞幹還原的單詞列表 type(tokens) 為一維list。

#查驗用
#for token in tokens:
    #print(token)

In [34]:
len(df_news['tfidftext'][12])

232

In [87]:
print(df_news.shape)

body_length = 0
tfidftext_length = 0
body_length_list = []
tfidftext_length_list = []
for i in range(df_news.shape[0]):
    
    a = df_news['body'][i]
    newstokens = nltk.tokenize.word_tokenize(a)
    body_length += len(newstokens)
    body_length_list.append(len(newstokens))
    
    tfidftext_length += len(df_news['tfidftext'][i])
    tfidftext_length_list.append(len(df_news['tfidftext'][i]))
    
print("平均文本長度為：",body_length//df_news.shape[0])    
print("去除停用詞之平均文本長度為：",tfidftext_length//df_news.shape[0])

(106496, 4)
平均文本長度為： 894
去除停用詞之平均文本長度為： 508


## 至此為止，單篇文章的處理顆粒度已到最小!

### 為了獲取TF-IDF的詞頻計算，我們需要將字詞 還原成text 以輸入tfidf模型

In [35]:
#建立一個新的dataframe來處理資料吧! -->還缺匯率漲跌的欄位後續再補。
date = []
word2vec_body = []
tfidf_body = []
bert_body = []
source_title = []

for newsbody_index in range(df_news.shape[0]):
    b = df_news['body'][newsbody_index]
    d = df_news['date'][newsbody_index]
    q = df_news['tfidftext'][newsbody_index]
    t = df_news['source_title'][newsbody_index]
    s = df_news['word_segmentation'][newsbody_index]
    word2vec_body.append(b)
    date.append(d)
    tfidf_body.append(" ".join(q))
    source_title.append(t)
    bert_body.append(s)
    
dic = {
    "date" : date,
    "word2vec_body" : word2vec_body,
    "tfidf_body" : tfidf_body,
    "bert_body" : bert_body,
    "source_title" : source_title
}

df = pd.DataFrame(dic)
df

,date,word2vec_body,tfidf_body,bert_body,source_title
0,2016-01-01,unemployment continues to fall inflation is in...,unemployment continue fall inflation check int...,[unemployment continues to fall inflation is i...,Forbes
1,2016-01-01,was a year of tremendous political and econom...,year tremendous political economic volatility ...,[ was a year of tremendous political and econo...,Forbes
2,2016-01-01,riverside co is ringing in the new year with i...,riverside co ring new year late acquisition re...,[riverside co is ringing in the new year with ...,The Wall Street Journal
3,2016-01-01,distrust in banking system hinders capital con...,distrust banking system hinders capital contro...,[distrust in banking system hinders capital co...,Bloomberg Business
4,2016-01-01,in the years since a peace treaty was signed i...,year since peace treaty sign belfast end three...,[in the years since a peace treaty was signed ...,International Business Times
...,...,...,...,...,...
101552,2020-12-31,i write about energy and commodities from rene...,write energy commodity renewable energy coal h...,[i write about energy and commodities from ren...,Forbes
101553,2020-12-31,philanthropic endeavors are nothing new to the...,philanthropic endeavor nothing new watch indus...,[philanthropic endeavors are nothing new to th...,MarketWatch
101554,2020-12-31,wall street indices finished at alltime highs ...,wall street index finish alltime high thursday...,[wall street indices finished at alltime highs...,International Business Times
101555,2020-12-31,the us dollar went from life preserver to cast...,u dollar go life preserver castaway topsyturvy...,[the us dollar went from life preserver to cas...,MarketWatch


In [36]:
# 儲存一下資料!!!
df.to_csv("./Pre_data/news/EURNews105109.csv")